## Primeros modelos KERAS sobre MNIST

Vamos a implementar las primeras redes neuronales para el conjunto de datos MNIST

IMPORTAMOS el Dataset y lo normalizamos:

In [ ]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Modelos secuenciales en KERAS

Los modelos secuenciales son modelos donde las capas se van añadiendo una tras otra. Te permite crear topologías básicas sin conexiones que no sean estrictamente lineales.

La red neuronal a crear es la siguiente:

- Capa de entrada de 784 neuronas, acorde con la dimensionalidad de los datos
- Capa oculta de 512 neuronas, con función de activación ReLU
- Capa de salida de 10 neuronas, acorde con el número de clases. Al ser un problema de clasificación emplearemos la función de activación Softmax

Este sería la definición del modelo:

In [ ]:
from keras import Sequential
from keras.layers import Dense, Input

model = Sequential()

model.add(Input(784))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

![model summary](summary.png)

El summary del modelo nos lista las diferentes capas del mismo así como su número de parámetros. Por ejemplo la capa densa oculta tiene 784 entradas x 512 salidas lo que implica una matriz de 784x512 = 401408 componentes. Además esta capa Dense tiene un vector de bias de 512 componentes, por lo tanto en total son 401408+512 = **401920**, que coincide con el número de parámetros de la tabla para dicha capa.

Podemos generar una imagen con la topología del modelo:

In [ ]:
keras.utils.plot_model(model, to_file="model.png")

## Compilar el modelo 

Para terminar la definición de nuestro modelo debemos definir dos componentes muy importantes:

1. Función de pérdida. En nuestro caso al ser un problema de clasificación emplearemos la **categorical_crossentropy**
2. Optimizador. En nuestro caso y para empezar emplearemos un sencillo descenso por gradiente estocástico **SGD**

opcionalmente:

3. Definir una métrica asociada a la calidad del modelo. En nuestro caso sería la tasa de acierto **accuracy**


Una vez definidos pasamos a compilar el modelo:

In [ ]:
from keras.optimizers import SGD

sgd=SGD(learning_rate=0.01, momentum=0.9)

# Compile Model
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



## Entrenar el modelo

Una vez tenemos los datos cargados y normalizados, así como el modelo ya compilado, podemos realizar el entrenamiento mediante el método **fit**. Para ello previamente necesitamos definir el tamaño del batch así como el número de epochs.

Al mismo tiempo que entrenamos el modelo con los datos de entrenamiento vamos a ir evaluando dicho modelo sobre los datos de test. Además, vamos generando un **history** con la evolución del modelo para luego poder crear gráficas del mismo.

In [ ]:
batch_size=32
epochs=25

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

## Explorar el History

Realizar gráfica con resultados de accuracy

In [ ]:
print(history.history.keys())

import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



Realizar gráfica con resultados de loss

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### **Ejercicio** 

probar diferentes entrenamientos dependiendo de los siguientes valores y anotar el accuracy en test alcanzado

| Dense/Batch  | 16  | 32  |  64 | 128  |
|---|---|---|---|---|
|  256  |   |   |   |   |
|  512  |   |   |   |   |
|  1024 |   |   |   |   |

# Salvar y cargar el modelo

Normalmente necesitaremos salvar el modelo entrenado para emplearlo más tarde en producción (inferencia)



In [ ]:
# Cargar datos y normalizar
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# crear modelo y entrenar
from keras import Sequential
from keras.layers import Dense, Input

model = Sequential()

model.add(Input(784))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

from keras.optimizers import SGD

sgd=SGD(learning_rate=0.01, momentum=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



batch_size=32
epochs=25

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))


# guardar modelo
model.save("model.keras")

# cargar modelo
from keras.models import load_model
model = load_model("model.keras")

# evaluar modelo
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
